In [1]:
import numpy as np
import pandas as pd
#William Connor Parham
#Dr. Hairong Qi
#COSC 522
#Project 1
#9/19/2023

**Read In Training and Testing Set**

In [8]:
#Read in the training and testing set
training_set = pd.read_csv('synth.tr', delim_whitespace=True)
testing_set = pd.read_csv('synth.te', delim_whitespace=True)
print(training_set)
print(testing_set)

         xs            ys       yc
0      0.05100797  0.16086164    0
1     -0.74807425  0.08904024    0
2     -0.77293371  0.26317168    0
3      0.21837360  0.12706142    0
4      0.37268336  0.49656200    0
..                             ...
245    0.26887846  0.44559230    1
246   -0.49254862  1.01443372    1
247    0.07615960  0.63795180    1
248    0.49226224  0.46876241    1
249   -0.40249641  0.71301084    1

[250 rows x 1 columns]
           xs        ys  yc
0   -0.970990  0.429425   0
1   -0.631997  0.251953   0
2   -0.773606  0.690751   0
3   -0.606212  0.175678   0
4   -0.539409  0.376744   0
..        ...       ...  ..
995  0.327675  0.608014   1
996  0.247590  0.279270   1
997  0.418515  1.044157   1
998  0.232315  0.819643   1
999  0.762041  0.573218   1

[1000 rows x 3 columns]


**K-Nearest Neighbor Implementation**

In [3]:
class KNN_Classifier:
    def __init__(self, k=1):
        self.n_neighbors = k

    def euclidian_distance(self, a, b):
        eucl_distance = 0.0

        for index in range(len(a)):
            eucl_distance += (a[index] - b[index] ** 2)
            euclidian_distance = np.sqrt(eucl_distance)
        
        return euclidian_distance


    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def predict_knn(self, X): 
        #initialize prediction_knn as empty list 
        prediction_knn = []

        for index in range(len(X)):
            #initialize euclidian distances as empty list
            euclidian_distances = []

            for row in self.X_train:
                #for every row in X_train, find eucl_distance to X using
                #euclidian_distance() and append to euclidian_distances list
                eucl_distance = self.euclidian_distance(row, X[index])
                euclidian_distances.append(eucl_distance)
            
            #sort euclidian_distances in ascending order, and retain only k
            #neighbors as specified in n_neighbors(n_neighbors=k)
            neighbors = np.array(euclidian_distances).argsort()[: self.n_neighbors]

            #initialize dict to count class occurrences in y_train
            count_neighbors = {}

            for val in neighbors:
                if self.y_train[val] in count_neighbors:
                    count_neighbors[self.y_train[val]] += 1
                else:
                    count_neighbors[self.y_tain[val]] = 1
                
            #max count labels to prediction_knn
            prediction_knn.append(max(count_neighbors, key=count_neighbors.get))
        
        return prediction_knn
    
    def display_knn(self, x):

        #initialize euclidian_distances as empty list
        euclidian_distances = []

        #for every row in X_train, find eucl_distance to x
        #using euclidian_distance() and append to euclidian_distances list
        for row in self.X_train:
            eucl_distance = self.euclidian_distance(row,x)
            euclidian_distances.append(eucl_distance)
        
        #sort euclidian-distances in ascneding order, and retain only k
        #neighbors as specified in n_neighbors (n_neighbors = k)
        neighbors = np.array(euclidian_distances).argsort()[: self.n_neighbors]

        #initiate empty display_knn_values list
        display_knn_values = []

        for index in range(len(neighbors)):
            neighbor_index = neighbors[index]
            e_distances = euclidian_distances[index]
            display_knn_values.append((neighbor_index, e_distances))
        
        return display_knn_values
